In [ ]:
import CarbonaraDataTools as CDT
import ipywidgets as widgets
from IPython.display import display, clear_output
import plotly.io as pio
pio.renderers.default='notebook'
import numpy as np
import os
import subprocess

In [ ]:
# <-- Press play to enter your file locations
pdbFile_text = widgets.Text(
    value='newFitData/test_simple/Lysozyme.pdb',
    placeholder='Enter your PDB file location:',
    description='Enter your PDB file location:',
    disabled=False,
    layout={'width': '550px','height':'50px'},   
)

pdbFile_text.style = {'description_width': '200px','description_height': '25px'}

def pdbFile_handler(change):
    global pdb_file
    pdb_file = change.new
    with output:
        clear_output(wait=True)

pdb_file = 'newFitData/test_simple/Lysozyme.pdb'
# Output widget
output = widgets.Output()
pdbFile_text.observe(pdbFile_handler,names='value')
display(pdbFile_text)

# your SAXS file location
saxsFile_text = widgets.Text(
    value='newFitData/test_simple/Saxs.dat',
    placeholder='Enter your SAXS file location:',
    description='Enter your SAXS file location:',
    disabled=False,
    layout={'width': '550px','height':'50px'},   
)

saxsFile_text.style = {'description_width': '200px','description_height': '25px'}

def saxsFile_handler(change):
    global SAXS_file
    global working_path
    SAXS_file = change.new
    working_path = os.path.dirname(SAXS_file)
    with output:
        clear_output(wait=True)

SAXS_file = 'newFitData/test_simple/Saxs.dat'
# Output widget
output = widgets.Output()
saxsFile_text.observe(saxsFile_handler,names='value')
display(saxsFile_text)

# a name for your prediction
molName_text = widgets.Text(
    value='TestLyso',
    placeholder='Enter a name for your prediction:',
    description='Enter a name for your prediction:',
    disabled=False,
    layout={'width': '550px','height':'50px'},   
)

molName_text.style = {'description_width': '200px','description_height': '25px'}

def molName_handler(change):
    global mol_name
    mol_name = change.new
    with output:
        clear_output(wait=True)

mol_name = 'TestLyso'
# Output widget
output = widgets.Output()
molName_text.observe(molName_handler,names='value')
display(molName_text)
working_path = 'newFitData/'+mol_name
confirm_filelocs_text = 'Confirm file locations'

confirm_filelocs_button = widgets.Button(
    description=confirm_filelocs_text,
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip=confirm_filelocs_text,
    layout=widgets.Layout(width=f'{len(confirm_filelocs_text) + 4}ch'), 
    icon='check' # (FontAwesome names without the `fa-` prefix)
)
output = widgets.Output()
confirm_filelocs_button.style = {'description_width': '300px','description_height': '25px'}

def confirm_filelocs_button_clicked(b):
    global working_path
    working_path = 'newFitData/'+mol_name
    if not os.path.isdir(working_path):
        os.makedirs(working_path)
    with output:
        clear_output(wait=True)
        print("File locations confirmed.")     


confirm_filelocs_button.on_click(confirm_filelocs_button_clicked)
display(confirm_filelocs_button)
display(output)

Make sure you've confirmed your file locations before running the next cell!

In [ ]:
# <-- Press play to check for any breaks in your backbone

coords_chains, sequence_chains, secondary_structure_chains, missing_residues_chains = CDT.pull_structure_from_pdb(pdb_file)
chains=[1]
# >> check for missing residues geometrically in the chain(s)
for coords in coords_chains:
    breaking_indices = CDT.missing_ca_check(coords)
    if len(breaking_indices) > 0:
        chains = [i+1 for i in range(len(breaking_indices)+1)]
        print("We think your PDB file has", len(breaking_indices)+1, 'chains')
        coords_chains, sequence_chains = CDT.break_into_chains(coords_chains[0],sequence_chains[0],breaking_indices)
    else:
        print("We think you've got a monomer")


# use kappa tau method to identify secondary structure
secondarysplit= [CDT.findSimilar(CDT.getKapTauList(coords_chains[i-1]),0.2) for i in chains]
# Writes all the coords, fingerprint, mixture & SAXS in Carbonara's expected input / file struct
CDT.write_fingerprint_file(len(coords_chains), sequence_chains, secondarysplit, working_path)    
CDT.write_coordinates_file(coords,working_path)
CDT.write_mixture_file(working_path)
CDT.write_saxs(SAXS_file, working_path)

varyingSection_tensor = [[] for i in range(len(breaking_indices)+1)]

# Initial SAXS fit

In [ ]:
# <-- Press play to view the initial fit to the scattering

script_name = CDT.write_initial_saxs_check_sh(working_path,mol_name,1,1)
subprocess.call(['sh', script_name],stdout=subprocess.DEVNULL,stderr=subprocess.STDOUT)
CDT.SAXS_fit_plotter(SAXS_file,working_path+'/tmp/mol1_initial_scatter.dat',False)

# Picking the q range

In [ ]:
# <-- Press play to enter your desired q range for the fit

minq_text = widgets.Text(
    value='0.01',
    placeholder='Enter your min q:',
    description='Enter your min q:',
    disabled=False,
    layout={'width': '250px','height':'50px'},   
)

minq_text.style = {'description_width': '150px','description_height': '25px'}

def minq_handler(change):
    global minq
    minq = change.new
    with output:
        clear_output(wait=True)

minq = '0.01'
# Output widget
output = widgets.Output()
minq_text.observe(minq_handler,names='value')
display(minq_text)

# <-- Press play to enter your SAXS file location
maxq_text = widgets.Text(
    value='0.25',
    placeholder='Enter your max q:',
    description='Enter your max q:',
    disabled=False,
    layout={'width': '250px','height':'50px'},   
)

maxq_text.style = {'description_width': '150px','description_height': '25px'}

def maxq_handler(change):
    global maxq
    maxq = change.new
    with output:
        clear_output(wait=True)

maxq = '0.25'
# Output widget
output = widgets.Output()
maxq_text.observe(maxq_handler,names='value')
display(maxq_text)

# Recommending varying sections

In [ ]:
# Running this cell will build the prediction for sections we think you should vary, view these below
allowed_linker, linker_indices = CDT.find_non_varying_linkers(working_path+'/coordinates1.dat',working_path+'/fingerPrint1.dat')
ss_len_tensor = [len(i) for i in CDT.get_sses(working_path+'/fingerPrint1.dat')]
# Calculate cumulative lengths
cumulative_lengths = np.cumsum(ss_len_tensor)

# Initialize an empty list to hold the chunked groups
chunked_groups = [[] for _ in range(len(cumulative_lengths))]

# Group the indices based on the cumulative lengths
for index in allowed_linker:
    for i, cum_length in enumerate(cumulative_lengths):
        if index < cum_length:
            chunked_groups[i].append(index)
            break

In [ ]:
# <-- Press play to select which chain to process

chains_Dropdown = widgets.Dropdown(
    options = chains,
    value = chains[0],
    description = 'Select which chain to process from the dropdown box ',
    disabled=False,
    layout={'width': '400px','height':'25px'}
)
chains_Dropdown.style = {'description_width': '350px','description_height': '25px'}

def chains_handler(change):
    global chain
    chain = change.new
    with output:
        clear_output(wait=True)  # Clear previous output

chain = chains[0]
chains_Dropdown.observe(chains_handler,names='value')
display(chains_Dropdown)


In the below cell, we highlight the sections we recommend to allow to vary during the run. Using ctrl + click, you can toggle these sections. 
* If you have changed the selected chain above, make sure to rerun this cell!

In [ ]:
#<--- Press the play button here to view the recommend varying sections
fullPoss = CDT.possibleLinkerList(pdb_file,working_path+'/fingerPrint1.dat',chain)
possLinkers = [i[1] for i in fullPoss]
if chain==1:
    recs = [i[1] for i in fullPoss if i[0] in allowed_linker.astype(str) and CDT.linkerLengthCheck(i[1])]
    chunked_groups[chain-1] = [int(i[0]) for i in fullPoss if i[0] in allowed_linker.astype(str) and CDT.linkerLengthCheck(i[1])]
    linker_SelectMultiple = widgets.SelectMultiple(
    options=possLinkers,
    value=recs,
    #rows=10,
    description='Varying Subsections',
    layout={'width': '300px','height':'150px'},
    disabled=False
    )

    def SelectMultiple_handler(change):
        global varyingLinkers
        varyingLinkers = change.new
        with output:
            clear_output(wait=True)
            selectedLinkers = [int(i[0]) for i in fullPoss if i[1] in varyingLinkers]
            chunked_groups[chain-1] = [i for i in selectedLinkers]
            CDT.highlightVaryingSections(working_path,pdb_file,selectedLinkers)

    linker_SelectMultiple.style = {'description_width': '150px','description_height': '25px'}

    # Output widget
    output = widgets.Output()
    recLinkers = [int(i[0]) for i in fullPoss if i[1] in recs]
    with output:
        CDT.highlightVaryingSections(working_path,pdb_file,recLinkers)
    linker_SelectMultiple.observe(SelectMultiple_handler,names='value')
    display(linker_SelectMultiple)
    display(output)
else:
    recs = [i[1] for i in fullPoss if int(i[0])+cumulative_lengths[chain-2] in allowed_linker and CDT.linkerLengthCheck(i[1])]
    chunked_groups[chain-1] = [int(i[0])+cumulative_lengths[chain-2] for i in fullPoss if int(i[0])+cumulative_lengths[chain-2] in allowed_linker and CDT.linkerLengthCheck(i[1])]
    linker_SelectMultiple = widgets.SelectMultiple(
    options=possLinkers,
    value=recs,
    #rows=10,
    description='Varying Subsections',
    layout={'width': '300px','height':'150px'},
    disabled=False
    )

    def SelectMultiple_handler(change):
        global varyingLinkers
        varyingLinkers = change.new
        with output:
            clear_output(wait=True)
            selectedLinkers = [int(i[0]) for i in fullPoss if i[1] in varyingLinkers]
            chunked_groups[chain-1] = [i+cumulative_lengths[chain-2] for i in selectedLinkers]
            CDT.highlightVaryingSections(working_path,pdb_file,selectedLinkers)

    linker_SelectMultiple.style = {'description_width': '150px','description_height': '25px'}

    # Output widget
    output = widgets.Output()
    recLinkers = [int(i[0]) for i in fullPoss if i[1] in recs]
    with output:
        CDT.highlightVaryingSections(working_path,pdb_file,recLinkers)
    linker_SelectMultiple.observe(SelectMultiple_handler,names='value')
    display(linker_SelectMultiple)
    display(output)



Once you have selected the varying sections for <b>all</b> chains in your molecule, run the below cell and click the button to confirm.

In [ ]:
# <-- Run this cell to confirm you are happy with your selected varying sections
varSec_text = 'Confirm varying sections'

varSec_button = widgets.Button(
    description='Confirm varying sections',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Confirm varying sections',
    layout=widgets.Layout(width=f'{len(varSec_text) + 4}ch'), 
    icon='check' # (FontAwesome names without the `fa-` prefix)
)
output = widgets.Output()
varSec_button.style = {'description_width': '300px','description_height': '25px'}

def on_varSec_button_clicked(b):
    print("Varying sections confirmed.")
    with output:
        clear_output(wait=True)
        flattened = [item for sublist in chunked_groups for item in sublist]
        CDT.write_varysections_file(flattened,working_path)
        
        
        
varSec_button.on_click(on_varSec_button_clicked)
display(varSec_button)
display(output)

# Distance constraints

In [ ]:
# <-- Run this cell to add distance constraints

resA_text = widgets.Text(
    value='',
    placeholder=' ',
    description='From resID:',
    disabled=False   
)

resB_text = widgets.Text(
    value='',
    placeholder=' ',
    description='To resID:',
    disabled=False   
)

dist_text = widgets.Text(
    value='',
    placeholder=' ',
    description="Distance (Angstroms):",
    disabled=False   
)

resA_text.style = {'description_width': '150px','description_height': '25px'}
resB_text.style = {'description_width': '150px','description_height': '25px'}
dist_text.style = {'description_width': '150px','description_height': '25px'}

contactPreds = []
fixedDistSet=[]

def resA_handler(change):
    global resA
    resA = change.new
    with output:
        clear_output(wait=True)

def resB_handler(change):
    global resB
    resB = change.new
    with output:
        clear_output(wait=True)

def dist_handler(change):
    global dist
    dist = change.new
    with output:
        clear_output(wait=True)

resA = ''
resB = ''
dist = ''

output = widgets.Output()

resA_text.observe(resA_handler,names='value')
resB_text.observe(resB_handler,names='value')
dist_text.observe(dist_handler,names='value')
display(resA_text)
display(resB_text)
display(dist_text)
### Add distance constraint button
distCons_text = 'Add distance constraint'
distCons_button = widgets.Button(
    description=distCons_text,
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip=distCons_text,
    layout=widgets.Layout(width=f'{len(distCons_text) + 4}ch'), 
    icon='check' # (FontAwesome names without the `fa-` prefix)
)

distCons_button.style = {'description_width': '300px','description_height': '25px'}

def on_distCons_button_clicked(b):
    with output:
        print('Distance constraint added')
        clear_output(wait=True)
        contactPreds.append([int(resA),int(resB)])
        fixedDistSet.append(float(dist))

### Check distance constraint button

check_distCons_text = 'Check distance constraints'
check_distCons_button = widgets.Button(
    description=check_distCons_text,
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip=check_distCons_text,
    layout=widgets.Layout(width=f'{len(check_distCons_text) + 4}ch'), 
    icon='check' # (FontAwesome names without the `fa-` prefix)
)

check_distCons_button.style = {'description_width': '300px','description_height': '25px'}

def check_distCons_button_clicked(b):
    with output:
        clear_output(wait=True)
        print("Distance constraints are: ")
        print(contactPreds)
        print(fixedDistSet)

### Reset distance constraint button

reset_distCons_text = 'Reset distance constraints'
reset_distCons_button = widgets.Button(
    description=reset_distCons_text,
    disabled=False,
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
    tooltip=reset_distCons_text,
    layout=widgets.Layout(width=f'{len(reset_distCons_text) + 4}ch'), 
    icon='check' # (FontAwesome names without the `fa-` prefix)
)

reset_distCons_button.style = {'description_width': '300px','description_height': '25px'}

def reset_distCons_button_clicked(b):
    print("Distance constraints reset.")
    global contactPreds
    global fixedDistSet
    with output:
        clear_output(wait=True)
        contactPreds = []
        fixedDistSet=[]
               
### Confirm distance constraint button   

confirm_distCons_text = 'Confirm distance constraints'
confirm_distCons_button = widgets.Button(
    description=confirm_distCons_text,
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip=confirm_distCons_text,
    layout=widgets.Layout(width=f'{len(confirm_distCons_text) + 4}ch'), 
    icon='check' # (FontAwesome names without the `fa-` prefix)
)

confirm_distCons_button.style = {'description_width': '300px','description_height': '25px'}

def confirm_distCons_button_clicked(b):
    print("Distance constraints confirmed.")
    with output:
        clear_output(wait=True)
        CDT.translate_distance_constraints(contactPreds,coords_chains[chain-1],working_path,fixedDistSet)
        
### Set on click actions               
distCons_button.on_click(on_distCons_button_clicked)
check_distCons_button.on_click(check_distCons_button_clicked)
reset_distCons_button.on_click(reset_distCons_button_clicked)   
confirm_distCons_button.on_click(confirm_distCons_button_clicked)

### Display all
display(distCons_button)
display(output)
display(check_distCons_button)
display(output)
display(reset_distCons_button)
display(output)
print('Once you have added all your distance constraints, confirm below')
display(confirm_distCons_button)
display(output)

# Write the sh file

In [ ]:
CDT.write_run_sh_file(working_path,mol_name,'testrun',1,minq,maxq,20,10)

In [ ]:
p = subprocess.Popen(['nohup','bash',script_name,'&'],stdout=subprocess.PIPE,stderr=subprocess.PIPE, preexec_fn=os.setpgrp)